In [1]:
"""
create_gmail_labels.py

Creates this structure exactly once:

Top-level:
  • Meet Request
  • Requires Attention
Parent + children:
  • Job Update
      - Application Incomplete
      - Applied
      - Assessment
      - Interview
      - Offer Made
      - Other
      - Rejected
      - Withdrawn
"""

from __future__ import annotations
import os
from typing import Dict, Iterable, List

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import sys
from pathlib import Path

# OAuth scope needed to create/modify labels
SCOPES = ["https://www.googleapis.com/auth/gmail.settings.basic"]

# Getting path of current file's parent directory
path = Path(os.path.dirname(os.getcwd()))
path = str(path)
sys.path.insert(1, path)

# ---------- Config you can tweak ----------
PARENT = "Job Update"
CHILDREN = [
    "Application Incomplete",
    "Applied",
    "Assessment",
    "Interview",
    "Offer Made",
    "Other",
    "Rejected",
    "Withdrawn",
]
TOP_LEVEL = ["Meet Request", "Requires Attention"]
# -----------------------------------------


def get_gmail_service():
    creds = Credentials.from_authorized_user_file(f'{path}/config/token.json')
    return build('gmail', 'v1', credentials=creds)


def list_labels(service) -> Dict[str, str]:
    """Return {name: id} for all existing labels."""
    resp = service.users().labels().list(userId="me").execute()
    return {lbl["name"]: lbl["id"] for lbl in resp.get("labels", [])}


def _create_label(service, name: str) -> str:
    """Create a label (visible in sidebar & message list). Return its ID."""
    body = {
        "name": name,
        "labelListVisibility": "labelShow",
        "messageListVisibility": "show",
    }
    created = service.users().labels().create(userId="me", body=body).execute()
    return created["id"]


def ensure_label(service, name: str) -> str:
    """Ensure a single label exists; return its ID."""
    existing = list_labels(service)
    if name in existing:
        return existing[name]
    try:
        return _create_label(service, name)
    except HttpError as e:
        if e.resp.status == 409:  # already exists (race)
            return list_labels(service)[name]
        raise


def ensure_label_tree(service, parent: str, children: Iterable[str]) -> Dict[str, str]:
    """
    Ensure the parent exists first, then each child as 'parent/child'.
    Returns {full_name: id}.
    """
    ids: Dict[str, str] = {}
    parent_id = ensure_label(service, parent)
    ids[parent] = parent_id

    # Now ensure each child under the parent (parent/child)
    for child in children:
        full_name = f"{parent}/{child}"
        ids[full_name] = ensure_label(service, full_name)
    return ids


def main():
    service = get_gmail_service()

    # Create top-level labels first (not nested)
    top_ids = {name: ensure_label(service, name) for name in TOP_LEVEL}

    # Create the parent, then nest the children under it
    tree_ids = ensure_label_tree(service, PARENT, CHILDREN)

    # Print a compact summary
    print("\n=== Top-level labels ===")
    for n, lid in top_ids.items():
        print(f"{n} -> {lid}")

    print("\n=== Parent + children ===")
    for n, lid in tree_ids.items():
        print(f"{n} -> {lid}")


if __name__ == "__main__":
    main()



=== Top-level labels ===
Meet Request -> Label_4718307430553739191
Requires Attention -> Label_7278646546916726427

=== Parent + children ===
Job Update -> Label_591402334336941725
Job Update/Application Incomplete -> Label_4552311577949953739
Job Update/Applied -> Label_3701903105754377425
Job Update/Assessment -> Label_6526643710159138362
Job Update/Interview -> Label_1176858450540225884
Job Update/Offer Made -> Label_682296045437184348
Job Update/Other -> Label_2627695902442400621
Job Update/Rejected -> Label_5388285104804611094
Job Update/Withdrawn -> Label_1877962796526733589
